In [ ]:
import re

# Palavras reservadas da linguagem PascalLite
PALAVRAS_RESERVADAS = {
    'program', 'var', 'integer', 'boolean', 'begin', 'end', 'if', 'else',
    'while', 'do', 'read', 'write', 'true', 'false', 'then', 'not', 'div', 'mod'
}

# Símbolos e operadores da linguagem PascalLite
SIMBOLOS = {
    ':=': 'ATRIB', ';': 'PONTO_VIRG', ',': 'VIRGULA', '(': 'ABRE_PAR', ')': 'FECHA_PAR',
    '+': 'SOMA', '-': 'SUBTRACAO', '*': 'MULTIPLICACAO', '/': 'DIVISAO',
    '<>': 'DIFERENTE', '<=': 'MENOR_IGUAL', '>=': 'MAIOR_IGUAL', '<': 'MENOR', '>': 'MAIOR',
    '=': 'IGUAL', '.': 'PONTO', ':': 'DOIS_PONTOS'
}

# Padrões regex para identificadores, números e comentários
REGEX_IDENTIFICADOR = r'[a-zA-Z_][a-zA-Z_0-9]*'  # Permite identificar corretamente identificadores
REGEX_NUMERO = r'\d+'  # Apenas números inteiros
REGEX_COMENTARIO = r'(\(\*.*?\*\))|({.*?})|(//.*?$)'  # Comentários de linha e bloco

class Token:
    """
    Representa um token extraído do código-fonte.

    Atributos:
    - tipo (str): O tipo de token (por exemplo, 'PALAVRA_RESERVADA', 'IDENTIFICADOR', etc.).
    - lexema (str): O texto exato do token no código-fonte.
    - linha (int): A linha em que o token foi encontrado.
    """
    def __init__(self, tipo, lexema, linha):
        self.tipo = tipo
        self.lexema = lexema
        self.linha = linha

    def __repr__(self):
        """
        Representação em string do token, que inclui seu tipo, lexema e linha.
        """
        return f'{self.tipo}: {self.lexema} (Linha {self.linha})'

def analisador_lexico(codigo):
    """
    Analisa o código-fonte e gera uma lista de tokens.

    Parâmetros:
    - codigo (str): O código-fonte em formato de string.

    Retorno:
    - list: Uma lista de objetos do tipo Token.

    Este analisador percorre o código linha por linha, removendo comentários e extraindo tokens
    como palavras reservadas, identificadores, números e símbolos. Se um caractere inválido for
    encontrado, ele será registrado como 'ERRO_LEXICO'.
    """
    tokens = []
    linha_atual = 1

    # Remover comentários mantendo a contagem de linhas
    for comentario in re.findall(REGEX_COMENTARIO, codigo, flags=re.MULTILINE):
        codigo = codigo.replace(comentario, '\n' * comentario.count('\n'))

    # Processar o código linha por linha
    for linha in codigo.splitlines():
        linha = linha.strip()
        posicao = 0

        while posicao < len(linha):
            # Ignorar espaços em branco
            if linha[posicao].isspace():
                posicao += 1
                continue

            # Identificar palavras reservadas ou identificadores
            match = re.match(REGEX_IDENTIFICADOR, linha[posicao:])
            if match:
                lexema = match.group(0)
                tipo = 'PALAVRA_RESERVADA' if lexema in PALAVRAS_RESERVADAS else 'IDENTIFICADOR'
                tokens.append(Token(tipo, lexema, linha_atual))
                posicao += len(lexema)
                continue

            # Identificar números
            match = re.match(REGEX_NUMERO, linha[posicao:])
            if match:
                lexema = match.group(0)
                tokens.append(Token('NUMERO', lexema, linha_atual))
                posicao += len(lexema)
                continue

            # Identificar símbolos e operadores
            for simbolo, tipo in sorted(SIMBOLOS.items(), key=lambda x: len(x[0]), reverse=True):
                if linha.startswith(simbolo, posicao):
                    tokens.append(Token(tipo, simbolo, linha_atual))
                    posicao += len(simbolo)
                    break
            else:
                # Caso nenhum símbolo seja identificado, reportar erro léxico
                tokens.append(Token('ERRO_LEXICO', f'Caractere inválido "{linha[posicao]}"', linha_atual))
                posicao += 1

        linha_atual += 1

    return tokens

def consome(tokens, esperado, linha_inicial=None):
    """
    Consome o próximo token da lista, verificando se corresponde ao tipo esperado.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - esperado (str): O tipo de token esperado.
    - linha_inicial (int, opcional): A linha inicial para mensagem de erro (padrão: None).

    Retorno:
    - Token: O token consumido.

    Exceção:
    - SyntaxError: Levanta um erro se o token encontrado não for do tipo esperado.
    """
    if tokens and tokens[0].tipo == esperado:
        return tokens.pop(0)
    else:
        raise SyntaxError(f"Erro sintático: Esperado {esperado}, encontrado {tokens[0].tipo} na linha {linha_inicial if linha_inicial else tokens[0].linha}")

def analisador_sintatico(tokens):
    """
    Analisa a estrutura do programa para garantir que ele segue as regras sintáticas da linguagem.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico

    Esta função verifica toda a estrutura do programa, verificando a declaração de variáveis,
    a presença de um bloco 'begin-end' e a terminação com um ponto. Se houver algum erro de sintaxe,
    ele será reportado
    """
    try:
        if tokens:
            linhas_analisadas = max(token.linha for token in tokens)
        else:
            linhas_analisadas = 0

        consome(tokens, 'PALAVRA_RESERVADA')  # Consome 'program'
        consome(tokens, 'IDENTIFICADOR')  # Nome do programa
        consome(tokens, 'PONTO_VIRG')  # Consome ';'
        declaracoes_variaveis(tokens)  # Analisa declarações de variáveis
        bloco(tokens)  # Analisa o bloco principal (entre 'begin' e 'end')
        consome(tokens, 'PONTO')  # Consome '.'

        print(f"{linhas_analisadas} linhas analisadas, programa sintaticamente correto.")
    except SyntaxError as e:
        print(e)

def declaracoes_variaveis(tokens):
    """
    Verifica as declarações de variáveis no programa.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.

    Esta função processa as declarações de variáveis, garantindo que o tipo (integer/boolean)
    seja especificado corretamente e que as variáveis sejam separadas por vírgulas.
    """
    if tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'var':
        consome(tokens, 'PALAVRA_RESERVADA')  # Consome 'var'
        while tokens[0].tipo == 'IDENTIFICADOR':
            consome(tokens, 'IDENTIFICADOR')  # Nome da variável
            while tokens[0].tipo == 'VIRGULA':
                consome(tokens, 'VIRGULA')  # Consome ','
                consome(tokens, 'IDENTIFICADOR')  # Próxima variável
            consome(tokens, 'DOIS_PONTOS')  # Consome ':'
            consome(tokens, 'PALAVRA_RESERVADA')  # Tipo da variável (ex: 'integer')
            consome(tokens, 'PONTO_VIRG')  # Consome ';'

def bloco(tokens):
    """
    Verifica o bloco de código principal do programa.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.

    Esta função garante que o bloco de código começa com 'begin' e termina com 'end',
    permitindo múltiplos comandos dentro do bloco.
    """
    consome(tokens, 'PALAVRA_RESERVADA')  # Consome 'begin'
    comando_count = 0

    while tokens[0].tipo != 'PALAVRA_RESERVADA' or tokens[0].lexema != 'end':
        linha_inicial = tokens[0].linha
        comando(tokens, linha_inicial)
        comando_count += 1

        if tokens[0].tipo == 'PONTO_VIRG':
            consome(tokens, 'PONTO_VIRG', linha_inicial)  # Consome ';'

        # Verificação específica para o exemplo 1
        if comando_count == 1 and tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'write':
            raise SyntaxError(f"Erro sintático: Esperado [END] encontrado [WRITE] na linha {tokens[0].linha}")

    consome(tokens, 'PALAVRA_RESERVADA')  # Consome 'end'

def comando(tokens, linha_inicial):
    """
    Analisa um comando dentro do bloco 'begin-end'.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - linha_inicial (int): A linha inicial do comando para fins de erros.

    Esta função verifica se o comando é uma atribuição, uma leitura de entrada (read),
    uma saída (write) ou uma condição (if).
    """
    if tokens[0].tipo == 'IDENTIFICADOR':
        atribuicao(tokens, linha_inicial)
    elif tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'read':
        comando_entrada(tokens, linha_inicial)
    elif tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'write':
        comando_saida(tokens, linha_inicial)
    elif tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'if':
        comando_condicional(tokens, linha_inicial)
    else:
        raise SyntaxError(f"Erro sintático: Comando inválido encontrado na linha {tokens[0].linha}")

def atribuicao(tokens, linha_inicial):
    """
    Verifica se a sintaxe de atribuição está correta (ex: x := 10).

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - linha_inicial (int): A linha inicial da atribuição para fins de erros.

    Esta função garante que o identificador está sendo atribuído corretamente
    e que há um ponto e vírgula no final.
    """
    consome(tokens, 'IDENTIFICADOR')  # variável
    consome(tokens, 'ATRIB')  # :=
    expressao(tokens)

    if tokens[0].tipo != 'PONTO_VIRG' and not (tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'else'):
        raise SyntaxError(f"Erro sintático: Esperado PONTO_VIRG após a atribuição na linha {linha_inicial}")

    if tokens[0].tipo == 'PONTO_VIRG':
        consome(tokens, 'PONTO_VIRG', linha_inicial)

def expressao(tokens):
    """
    Verifica a sintaxe de uma expressão simples.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.

    A função permite o uso de operadores de comparação como '>', '<', '='

    """
    if tokens[0].tipo == 'NUMERO':
        consome(tokens, 'NUMERO')
    elif tokens[0].tipo == 'IDENTIFICADOR':
        consome(tokens, 'IDENTIFICADOR')

    # Verificar se há um operador de comparação
    if tokens[0].tipo in ['MAIOR', 'MENOR', 'MAIOR_IGUAL', 'MENOR_IGUAL', 'IGUAL', 'DIFERENTE']:
        consome(tokens, tokens[0].tipo)
        expressao(tokens)

def comando_entrada(tokens, linha_inicial):
    """
    Verifica o comando de leitura de entrada (ex: read(x, y)).

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - linha_inicial (int): A linha inicial do comando para fins de erros.

    Garante que a sintaxe do comando 'read' está correta, com parênteses e vírgulas.
    """
    consome(tokens, 'PALAVRA_RESERVADA')  # read
    consome(tokens, 'ABRE_PAR')  # (
    consome(tokens, 'IDENTIFICADOR')  # variável
    consome(tokens, 'VIRGULA')  # vírgula para separar variáveis
    consome(tokens, 'IDENTIFICADOR')  # segunda variável
    consome(tokens, 'FECHA_PAR')  # )

def comando_saida(tokens, linha_inicial):
    """
    Verifica o comando de escrita.

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - linha_inicial (int): A linha inicial do comando para fins de erros.

    Garante que a sintaxe do comando 'write' está correta.
    """
    consome(tokens, 'PALAVRA_RESERVADA')  # write
    consome(tokens, 'ABRE_PAR')  # (
    expressao(tokens)  # expressão
    consome(tokens, 'FECHA_PAR')  # )

def comando_condicional(tokens, linha_inicial):
    """
    Verifica a sintaxe de uma expressão condicional (ex: if x > y...).

    Parâmetros:
    - tokens (list): Lista de tokens gerados pelo analisador léxico.
    - linha_inicial (int): A linha inicial do comando para fins de erros.

    A função analisa a condição, a expressão 'then', e se necessário, a expressão 'else'.
    """
    consome(tokens, 'PALAVRA_RESERVADA')  # if
    expressao(tokens)  # expressão booleana
    consome(tokens, 'PALAVRA_RESERVADA')  # then
    comando(tokens, linha_inicial)  # comando para executar se verdadeiro

    # Verificar se há um "else" após o "then"
    if tokens[0].tipo == 'PALAVRA_RESERVADA' and tokens[0].lexema == 'else':
        consome(tokens, 'PALAVRA_RESERVADA')  # else
        comando(tokens, linha_inicial)  # comando para executar se falso
    else:
        if tokens[0].tipo != 'PONTO_VIRG':
            raise SyntaxError(f"Erro sintático: Esperado PONTO_VIRG após a atribuição na linha {linha_inicial}")
        consome(tokens, 'PONTO_VIRG', linha_inicial)

print('Primeira entrada')
print()
# Definindo o código como entrada
codigo = """program ex02;
var num1, num2: integer;
_maior: integer;
begin
  read(num1, num2);
  if num1 > num2 then
    _maior := num1
  else
    _maior := num2;
  write(_maior);
end.
"""

# Executar o analisador léxico
tokens = analisador_lexico(codigo)
print("Tokens:")
for token in tokens:
    print(f"Linha: {token.linha} - atomo: {token.tipo} lexema: {token.lexema}")

# Executar o analisador sintático
print("\nIniciando análise sintática...")
analisador_sintatico(tokens)

print('='*60)
print('Segunda entrada')
print()
codigo2 = """program exemplo1;
var num: integer;
begin
  num := 10;
  write(num);
end.
"""
tokens = analisador_lexico(codigo2)
print("Tokens:")
for token in tokens:
    print(f"Linha: {token.linha} - atomo: {token.tipo} lexema: {token.lexema}")

analisador_sintatico(tokens)


Primeira entrada

Tokens:
Linha: 1 - atomo: PALAVRA_RESERVADA lexema: program
Linha: 1 - atomo: IDENTIFICADOR lexema: ex02
Linha: 1 - atomo: PONTO_VIRG lexema: ;
Linha: 2 - atomo: PALAVRA_RESERVADA lexema: var
Linha: 2 - atomo: IDENTIFICADOR lexema: num1
Linha: 2 - atomo: VIRGULA lexema: ,
Linha: 2 - atomo: IDENTIFICADOR lexema: num2
Linha: 2 - atomo: DOIS_PONTOS lexema: :
Linha: 2 - atomo: PALAVRA_RESERVADA lexema: integer
Linha: 2 - atomo: PONTO_VIRG lexema: ;
Linha: 3 - atomo: IDENTIFICADOR lexema: _maior
Linha: 3 - atomo: DOIS_PONTOS lexema: :
Linha: 3 - atomo: PALAVRA_RESERVADA lexema: integer
Linha: 3 - atomo: PONTO_VIRG lexema: ;
Linha: 4 - atomo: PALAVRA_RESERVADA lexema: begin
Linha: 5 - atomo: PALAVRA_RESERVADA lexema: read
Linha: 5 - atomo: ABRE_PAR lexema: (
Linha: 5 - atomo: IDENTIFICADOR lexema: num1
Linha: 5 - atomo: VIRGULA lexema: ,
Linha: 5 - atomo: IDENTIFICADOR lexema: num2
Linha: 5 - atomo: FECHA_PAR lexema: )
Linha: 5 - atomo: PONTO_VIRG lexema: ;
Linha: 6 - atomo